In [32]:
from collections import namedtuple
import dataclasses as dc
import pathlib
import pprint
import random
import uuid

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pendulum
import seaborn as sns

In [34]:
from orchid import (project_store as ops)

In [35]:
import toolz.curried as toolz

In [36]:
# noinspection PyUnresolvedReferences
import UnitsNet
# noinspection PyUnresolvedReferences
from System import DateTime

In [37]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [38]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), 
                                  project_filenames)
project_path_names

{'bakken': WindowsPath('c:/src/Orchid.IntegrationTestData/frankNstein_Bakken_UTM13_FEET.ifrac'),
 'montney': WindowsPath('c:/src/Orchid.IntegrationTestData/Project-frankNstein_Montney_UTM13_METERS.ifrac')}

In [39]:
projects = toolz.valmap(
    lambda pn: ops.ProjectStore(str(pn)).native_project(),
    project_path_names)
projects

{'bakken': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x0000028E0A206E50>,
 'montney': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x0000028E249B11C0>}

In [40]:
def get_project_monitors(proj):
    return proj.Monitors.Items

project_monitors = toolz.pipe(
    projects,
    toolz.valmap(get_project_monitors),
)
project_monitors

{'bakken': <Orchid.FractureDiagnostics.IMonitor[] object at 0x0000028E227E23D0>,
 'montney': <Orchid.FractureDiagnostics.IMonitor[] object at 0x0000028E227E2460>}

In [41]:
toolz.pipe(project_monitors.values(),
           toolz.map(len),
           list,)

[13, 2]

In [42]:
@dc.dataclass
class MonitorBasics:
    object_id: uuid.UUID
    name: str
    display_name: str
    start_time: pendulum.DateTime
    stop_time: pendulum.DateTime

@toolz.curry
def summarize_monitor_basics(m):
    result = MonitorBasics(m.ObjectId, m.DisplayName, m.Name, 
                           m.StartTime.ToString('o'), m.StopTime.ToString('o'))
    return result

project_monitor_summaries = toolz.pipe(
    project_monitors,
    toolz.valmap(toolz.map(summarize_monitor_basics)),
    toolz.valmap(list),
)
project_monitor_summaries

{'bakken': [MonitorBasics(object_id=<System.Guid object at 0x0000028E227EF940>, name='Demo_2H - stage 1', display_name='Demo_2H', start_time='2018-06-06T06:57:39.0728704Z', stop_time='2018-06-08T18:41:57.6818816Z'),
  MonitorBasics(object_id=<System.Guid object at 0x0000028E227EF2B0>, name='Demo_4H - stage 6', display_name='Demo_4H', start_time='2018-06-08T10:01:14.0478464Z', stop_time='2018-06-12T08:31:14.5422336Z'),
  MonitorBasics(object_id=<System.Guid object at 0x0000028E227EF8B0>, name='Demo_1H - stage 8', display_name='Demo_1H', start_time='2018-06-08T14:30:09.6954368Z', stop_time='2018-06-12T12:40:58.0000000Z'),
  MonitorBasics(object_id=<System.Guid object at 0x0000028E227EFF10>, name='Demo_2H - stage 14', display_name='Demo_2H', start_time='2018-06-12T02:58:49.9896192Z', stop_time='2018-06-16T10:44:37.1484416Z'),
  MonitorBasics(object_id=<System.Guid object at 0x0000028E227EFAF0>, name='Demo_4H - stage 15', display_name='Demo_4H', start_time='2018-06-16T04:46:26.0778752Z', s

In [43]:
selected_field_name = 'bakken'

In [44]:
def make_id_summary(field_name, summaries):
    field_name_summaries = summaries[field_name]
    return {
        'field': field_name,
        'Object Id': [s.object_id for s in field_name_summaries],
        # 'Display Name': [s.display_name for s in field_name_summaries],
        # 'Name': [s.name for s in field_name_summaries],
        'Start Time': [s.start_time for s in field_name_summaries],
        'Stop Time': [s.stop_time for s in field_name_summaries],
    }

In [45]:
pd.DataFrame(data=make_id_summary('bakken', project_monitor_summaries))

,field,Object Id,Start Time,Stop Time
0,bakken,5b68d8c4-a578-44e7-bc08-b1d83483c4ec,2018-06-06T06:57:39.0728704Z,2018-06-08T18:41:57.6818816Z
1,bakken,4116e3d3-b1ba-4063-b41e-467c5c00eb20,2018-06-08T10:01:14.0478464Z,2018-06-12T08:31:14.5422336Z
2,bakken,8fab7763-8cad-42f4-8d44-899f2e8691bc,2018-06-08T14:30:09.6954368Z,2018-06-12T12:40:58.0000000Z
3,bakken,8660a506-e2a3-4427-8a03-d20e60c214df,2018-06-12T02:58:49.9896192Z,2018-06-16T10:44:37.1484416Z
4,bakken,9d702765-5696-4b38-a54c-84813898f907,2018-06-16T04:46:26.0778752Z,2018-06-19T01:00:07.3059072Z
5,bakken,182fa5d0-5695-40e8-ad59-ed18e796ee9c,2018-06-16T08:05:09.6496640Z,2018-06-19T04:36:34.6289024Z
6,bakken,c1d35d86-a8a1-4e46-a303-f2f1011a399f,2018-06-18T21:54:39.0344192Z,2018-06-22T18:31:17.2888192Z
7,bakken,be89b07b-e37f-4222-9759-acd5682dc7a0,2018-06-22T13:05:04.2663552Z,2018-06-25T08:51:22.6538112Z
8,bakken,14607f23-95f4-4405-b34b-daa0f924c2be,2018-06-22T16:24:09.5910656Z,2018-06-25T11:39:06.6796928Z
9,bakken,6777b2fe-7575-4fed-a82a-bb0b0085152d,2018-06-25T05:46:56.1804160Z,2018-06-29T00:10:52.0385792Z


In [46]:
pd.DataFrame(data=make_id_summary('montney', project_monitor_summaries))

,field,Object Id,Start Time,Stop Time
0,montney,ad002ac7-c2e5-44d4-b3cc-fca9ad84d10e,2018-04-06T16:44:00.0000000Z,2018-04-10T03:20:00.0000000Z
1,montney,34fbef8a-ba84-4437-bfe4-0b78a3e12199,2018-04-06T13:30:00.0000000Z,2018-04-09T22:06:00.0000000Z


In [47]:
@dc.dataclass
class PyTick:
    sample_at: DateTime
    sample_value: float
        
MonitorSummary = namedtuple('MonitorSummary', ['object_id',
                                               'name', 
                                               'display_name',
                                               'stage_no',
                                               'start',
                                               'stop',
                                               'time_series_id',
                                               'quantity_name',
                                               'sample_count',
                                               'samples'])
        
def monitors_time_series(ms):
    result = toolz.pipe(
        ms,
        toolz.map(lambda m:  MonitorSummary(
            object_id=m.ObjectId.ToString(), 
            name=m.Name, 
            display_name=m.DisplayName,
            stage_no=m.Stage.DisplayStageNumber if 'Stage' in m.GetType().Name else 'N/A',
            start=m.StartTime.ToString('s'),
            stop=m.StopTime.ToString('s'),
            time_series_id=str(m.TimeSeries.ObjectId), 
            quantity_name=m.TimeSeries.SampledQuantityName,
            sample_count=len(m.TimeSeries.GetOrderedTimeSeriesHistory()), 
            samples=m.TimeSeries.GetOrderedTimeSeriesHistory())),
        lambda summary: sorted(summary, key=lambda ms: ms.display_name),
        list,
    )
    return result

project_monitor_time_series = toolz.pipe(
    project_monitors,
    toolz.valmap(monitors_time_series),
    toolz.valmap(toolz.map(lambda ms: {ms.display_name: ms})),
    toolz.valmap(toolz.merge),
)
pprint.pprint(project_monitor_time_series)

{'bakken': {'Demo_1H - stage 22': MonitorSummary(object_id='182fa5d0-5695-40e8-ad59-ed18e796ee9c', name='Demo_1H', display_name='Demo_1H - stage 22', stage_no=22, start='2018-06-16T08:05:09', stop='2018-06-19T04:36:34', time_series_id='ed590275-68e9-4001-8934-40386dfd1472', quantity_name='Pressure', sample_count=114595, samples=<Orchid.FractureDiagnostics.TimeSeries.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]][] object at 0x0000028E227DA940>),
            'Demo_1H - stage 36': MonitorSummary(object_id='14607f23-95f4-4405-b34b-daa0f924c2be', name='Demo_1H', display_name='Demo_1H - stage 36', stage_no=36, start='2018-06-22T16:24:09', stop='2018-06-25T11:39:06', time_series_id='ed590275-68e9-4001-8934-40386dfd1472', quantity_name='Pressure', sample_count=114595, samples=<Orchid.FractureDiagnostics.TimeSeries.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]][] object at 0x0000028E2CAF0190>),
            'Demo_1H - stage 50': MonitorSummary(object_id='5e51285b-6ac9-4a23-a360-f5

In [48]:
def shuffle_em(es):
    random.shuffle(es)
    return es

unshuffled_sample_stages = {
    'Demo_1H': [8, 22, 36, 50] * 2,
    'Demo_2H': [1, 14, 29, 43] * 2,
    'Demo_4H': [6, 15, 25, 35] * 2,
}
sample_stages = toolz.merge(
    toolz.valmap(shuffle_em, unshuffled_sample_stages),
    {'Hori_02': [1] * 8,
     'Vert_01': [1] * 8}
)
sample_stages

{'Demo_1H': [8, 8, 50, 50, 22, 36, 22, 36],
 'Demo_2H': [29, 43, 43, 29, 14, 14, 1, 1],
 'Demo_4H': [25, 6, 35, 15, 6, 15, 25, 35],
 'Hori_02': [1, 1, 1, 1, 1, 1, 1, 1],
 'Vert_01': [1, 1, 1, 1, 1, 1, 1, 1]}

In [49]:
sample_stages = {
    'Demo_1H': [22, 50, 22, 8, 36, 50, 36, 8],
    'Demo_2H': [29, 14, 1, 29, 1, 14, 43, 43],
    'Demo_3H': ['MonitorWell'] * 8,
    'Demo_4H': [35, 25, 6, 35, 15, 15, 25, 6],
    'Hori_02': [1] * 8,
    'Vert_01': [1] * 8,
}

In [50]:
sample_indices = {
    'Demo_1H': [0, 6078, 24489, 35902, 47331, 56567, 94551, 114594],
    'Demo_2H': [0, 1105, 14475, 18418, 33263, 83966, 87110, 114594],
    'Demo_3H': [0, 9872, 38744, 47718, 58806, 58869, 62026, 62026],
    'Demo_4H': [0, 9872, 38744, 47718, 58806, 58869, 62026, 62757],
    'Hori_02': [0, 14848, 67025, 138193, 196604, 252497, 256716, 325041],
    'Vert_01': [0, 6652, 90175, 134841, 211515, 222007, 300671, 322124],
}

In [51]:
toolz.pipe(
    project_monitor_time_series,
    # toolz.get_in(['bakken', 'Demo_1H - stage 50']),
    # toolz.get_in(['bakken', 'Demo_2H - stage 1']),
    # toolz.get_in(['bakken', 'Demo_3H - MonitorWell']),
    # toolz.get_in(['bakken', 'Demo_4H - stage 25']),
    toolz.get_in(['montney', 'Hori_02 - 0 - stage 1']),
    # lambda ms: toolz.get(sample_indices['Demo_1H'], ms.samples),
    # lambda ms: toolz.get(sample_indices['Demo_2H'], ms.samples),
    # lambda ms: toolz.get(sample_indices['Demo_3H'], ms.samples),
    # lambda ms: toolz.get(sample_indices['Demo_4H'], ms.samples),
    lambda ms: toolz.get(sample_indices['Hori_02'], ms.samples),
    toolz.map(lambda tick: PyTick(sample_at=tick.Timestamp.ToString('s'), 
                                  sample_value=tick.Value)),
    list,
)

[PyTick(sample_at='2018-04-05T09:26:03', sample_value=16136.96875),
 PyTick(sample_at='2018-04-05T13:37:11', sample_value=15879.14063),
 PyTick(sample_at='2018-04-06T04:19:06', sample_value=15373.36328),
 PyTick(sample_at='2018-04-07T00:21:01', sample_value=24102.6582),
 PyTick(sample_at='2018-04-07T16:47:21', sample_value=16848.8125),
 PyTick(sample_at='2018-04-08T08:30:30', sample_value=14907.30176),
 PyTick(sample_at='2018-04-08T09:42:19', sample_value=14807.57031),
 PyTick(sample_at='2018-04-09T04:58:44', sample_value=13633.97754)]

In [62]:
def sample_monitor_summary(monitor_summary):
    result = toolz.assoc(
        monitor_summary,
        'samples',
        toolz.pipe(
            toolz.get(sample_indices[monitor_summary['name']], monitor_summary['samples'])),
#             toolz.map(lambda tick: PyTick(sample_at=tick.Timestamp.ToString('s'), 
#                                   sample_value=tick.Value)),
       )
    return result
        

toolz.pipe(
    project_monitor_time_series,
    toolz.valmap(toolz.valmap(lambda ms: ms._asdict())),
    toolz.valmap(toolz.valmap(sample_monitor_summary)),
)

{'bakken': {'Demo_1H - stage 22': {'object_id': '182fa5d0-5695-40e8-ad59-ed18e796ee9c',
   'name': 'Demo_1H',
   'display_name': 'Demo_1H - stage 22',
   'stage_no': 22,
   'start': '2018-06-16T08:05:09',
   'stop': '2018-06-19T04:36:34',
   'time_series_id': 'ed590275-68e9-4001-8934-40386dfd1472',
   'quantity_name': 'Pressure',
   'sample_count': 114595,
   'samples': (<Orchid.FractureDiagnostics.Factories.Implementations.TimeSeries.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]] object at 0x0000028E249B42E0>,
    <Orchid.FractureDiagnostics.Factories.Implementations.TimeSeries.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]] object at 0x0000028E249B4FA0>,
    <Orchid.FractureDiagnostics.Factories.Implementations.TimeSeries.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]] object at 0x0000028E2498CB80>,
    <Orchid.FractureDiagnostics.Factories.Implementations.TimeSeries.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]] object at 0x0000028E2498C550>,
    <Orchid.Fr